# Import Important Libraries

In [1]:
import pandas as pd          # for data manipulation and analysis
import math                  # for mathematical calculations

# Load The Dataset

In [2]:
order_report = pd.read_excel("C:\Projects\COINTAB\Assignment_Data\Company X - Order Report.xlsx")   # Website order report
invoice_data = pd.read_excel("C:\Projects\COINTAB\Assignment_Data\Courier Company - Invoice.xlsx")  # Courier company invoice
SKU = pd.read_excel("C:\Projects\COINTAB\Assignment_Data\Company X - SKU Master.xlsx")      # SKU master
pincode_zones = pd.read_excel("C:\Projects\COINTAB\Assignment_Data\Company X - Pincode Zones.xlsx")   # Warehouse pincode
weight_slab = pd.read_excel("C:\Projects\COINTAB\Assignment_Data\Courier Company - Rates.xlsx")    # Courier charges rate card at weight slab and pincode level

# Analysis

In [3]:
print(SKU.shape)    # There is duplicates in this dataset
print(SKU['SKU'].nunique()) 
SKU_unique = SKU.drop_duplicates()   # Remove the duplicates
print(SKU_unique.shape)
print(SKU_unique['SKU'].nunique())   # There is no duplicates in this modified dataset

(66, 2)
65
(65, 2)
65


In [4]:
order_report.shape            # Shpe of the company order report dataset

(400, 5)

In [5]:
# We will merge the Companies Report dataset to the SKU master dataset using the SKU column key to calculate the weight of orders.
order_SKU = pd.merge(left = order_report, 
                     right = SKU_unique,
                     how= 'inner',
                     on = 'SKU')

In [6]:
# Calculate the weught for each order

order_SKU['Total Weight X'] = order_SKU['Order Qty']*order_SKU['Weight (g)']
order_SKU['Total Weight X(KG)'] = order_SKU['Total Weight X']/1000
# order_SKU.head(5)

In [7]:
order_SKU['Total Price X'] = order_SKU['Order Qty']*order_SKU['Item Price(Per Qty.)']    # Calculate the price of each order

In [8]:
order_SKU.drop(['SKU', 'Order Qty', 'Item Price(Per Qty.)', 'Weight (g)', 'Total Weight X'], axis = 1, inplace = True)   # Drop the unwanted columns

In [9]:
order_SKU.head(2)

,ExternOrderNo,Payment Mode,Total Weight X(KG),Total Price X
0,2001827036,COD,0.127,233
1,2001821995,Prepaid,0.127,233


In [10]:
order_SKU.rename(columns = {'ExternOrderNo':'Order ID', 'Total Weight X(KG)' : 'Total Weight X'}, inplace = True)   # Rename the columns name

In [11]:
# There are duplicated Order ID so we will group them and sum the weight and price for each unique Order ID

groups = order_SKU.groupby(by = ['Order ID', 'Payment Mode']).sum()

In [12]:
order_X = groups.reset_index()

In [13]:
print(order_X.shape)    # There is only unique Order IDs
order_X.head(2)

(124, 4)


,Order ID,Payment Mode,Total Weight X,Total Price X
0,2001806210,COD,0.22,1324
1,2001806226,COD,0.48,2250


In [14]:
# Merge the order_X dataset to the Courier company invoice dataset on the Order ID column

order_courier = pd.merge(left = order_X, 
                         right = invoice_data,
                         how= 'inner',
                         on = 'Order ID')

In [15]:
order_courier.head(2)

,Order ID,Payment Mode,Total Weight X,Total Price X,AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,2001806210,COD,0.22,1324,1091117221940,2.92,121003,140604,b,Forward charges,174.5
1,2001806226,COD,0.48,2250,1091117222065,0.68,121003,723146,d,Forward charges,90.2


In [16]:
# Reorder some columns

second_column = order_courier.pop('AWB Code')
order_courier.insert(1, 'AWB Number', second_column)

third_column = order_courier.pop('Total Weight X')
order_courier.insert(2, 'Total weight as per X (KG)', third_column)

fourth_column = order_courier.pop('Charged Weight')
order_courier.insert(3, 'Total weight as per Courier Company (KG)', fourth_column)

fifth_column = order_courier.pop('Zone')
order_courier.insert(4, 'Delivery Zone charged by Courier Company', fifth_column)

In [17]:
pincode_zones.head(3)         # Analyse the warehouse pincode dataset

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d


In [18]:
print(pincode_zones.shape)
pincode_zones['Customer Pincode'].nunique()           # duplicates in the pincode dataset
pincode_zone = pincode_zones.drop_duplicates()

(124, 3)


In [19]:
# merge order_courier and pincode to find the delivery zone according to company X

order_courier = pd.merge(left = order_courier, 
                         right = pincode_zone,
                         how= 'inner',
                         on = 'Customer Pincode')

In [20]:
order_courier.head(2)

,Order ID,AWB Number,Total weight as per X (KG),Total weight as per Courier Company (KG),Delivery Zone charged by Courier Company,Payment Mode,Total Price X,Warehouse Pincode_x,Customer Pincode,Type of Shipment,Billing Amount (Rs.),Warehouse Pincode_y,Zone
0,2001806210,1091117221940,0.22,2.92,b,COD,1324,121003,140604,Forward charges,174.5,121003,b
1,2001806226,1091117222065,0.48,0.68,d,COD,2250,121003,723146,Forward charges,90.2,121003,d


In [21]:
weight_slab['Zone'] = weight_slab['Zone'].str.lower()
weight_slab

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,a,0.25,29.5,23.6,13.6,23.6
1,b,0.50,33.0,28.3,20.5,28.3
2,c,0.75,40.1,38.9,31.9,38.9
3,d,1.25,45.4,44.8,41.3,44.8
4,e,1.50,56.6,55.5,50.7,55.5


In [22]:
# merge order_courier and weight slab to find the weight slab according to company X

order_courier = pd.merge(left = order_courier, 
                         right = weight_slab,
                         how= 'inner',
                         on = 'Zone')

In [23]:
# Reorder some columns in order courier
sixth_column = order_courier.pop('Weight Slabs')
order_courier.insert(3, 'Weight slab as per X (KG)', sixth_column)

sixth_column = order_courier.pop('Zone')
order_courier.insert(4, 'Delivery Zone as per X', sixth_column)

order_courier.head(3)


,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Delivery Zone as per X,Total weight as per Courier Company (KG),Delivery Zone charged by Courier Company,Payment Mode,Total Price X,Warehouse Pincode_x,Customer Pincode,Type of Shipment,Billing Amount (Rs.),Warehouse Pincode_y,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,2001806210,1091117221940,0.220,0.5,b,2.92,b,COD,1324,121003,140604,Forward charges,174.5,121003,33.0,28.3,20.5,28.3
1,2001806233,1091117222135,0.245,0.5,b,0.78,b,Prepaid,1486,121003,263139,Forward charges,61.3,121003,33.0,28.3,20.5,28.3
2,2001806304,1091117222360,0.500,0.5,b,0.71,d,Prepaid,1082,121003,302017,Forward charges,90.2,121003,33.0,28.3,20.5,28.3


In [24]:
order_courier['Total weight as per X (KG)'] = order_courier['Total weight as per X (KG)'].round(2)

In [25]:
# test = order_courier[order_courier['Order ID'] == 2001808295]

In [26]:
# test

In [27]:
# Function to calculate the expected charge as per company X

def calculate_charges(df):
    df = df.copy()
    # print(df)
    num_slabs = math.ceil(df['Total weight as per X (KG)']/df['Weight slab as per X (KG)'])
    charges_fwd = df['Forward Fixed Charge'] + (num_slabs-1)*df['Forward Additional Weight Slab Charge']
    charges_rto = df['RTO Fixed Charge'] + (num_slabs-1)*df['RTO Additional Weight Slab Charge']
    
    
    if df['Type of Shipment'] == 'Forward charges':
        return charges_fwd  
    else:
        return charges_fwd + charges_rto 

In [28]:
order_courier['Expected Charge as per X (Rs.)'] = order_courier.apply(calculate_charges, axis = 1)   # Apply the function

In [29]:
# Calculate the difference

order_courier['Difference Between Expected Charges and Billed Charges (Rs.)'] = order_courier['Expected Charge as per X (Rs.)'] - order_courier['Billing Amount (Rs.)']

In [30]:
# Total Orders where X has been overcharged
total_over = (order_courier['Difference Between Expected Charges and Billed Charges (Rs.)'] < 0)
count_over = total_over.sum()

# Total Orders where X has been undercharged
total_under = (order_courier['Difference Between Expected Charges and Billed Charges (Rs.)'] > 0)
count_under = total_under.sum()

# Total orders where X has been correctly charged
total_equal = (order_courier['Difference Between Expected Charges and Billed Charges (Rs.)'] == 0)
count_equal = total_equal.sum()

# Print all values
print(count_under)
print(count_over)
print(count_equal)

7
103
14


In [31]:
# calculate the total amount for each scenario

total_equal_invoice_amount = round(order_courier[total_equal]['Billing Amount (Rs.)'].sum(), 1)
print(total_equal_invoice_amount)

total_over_invoice_amount = round(order_courier[total_over]['Difference Between Expected Charges and Billed Charges (Rs.)'].sum(),1)
print(total_over_invoice_amount)

total_under_invoice_amount = round(order_courier[total_under]['Difference Between Expected Charges and Billed Charges (Rs.)'].sum(),1)
print(total_under_invoice_amount)

863.6
-6696.3
103.2


In [32]:
order_courier.columns

Index(['Order ID', 'AWB Number', 'Total weight as per X (KG)',
       'Weight slab as per X (KG)', 'Delivery Zone as per X',
       'Total weight as per Courier Company (KG)',
       'Delivery Zone charged by Courier Company', 'Payment Mode',
       'Total Price X', 'Warehouse Pincode_x', 'Customer Pincode',
       'Type of Shipment', 'Billing Amount (Rs.)', 'Warehouse Pincode_y',
       'Forward Fixed Charge', 'Forward Additional Weight Slab Charge',
       'RTO Fixed Charge', 'RTO Additional Weight Slab Charge',
       'Expected Charge as per X (Rs.)',
       'Difference Between Expected Charges and Billed Charges (Rs.)'],
      dtype='object')

In [33]:
order_courier.drop(columns = ['Payment Mode', 'Total Price X', 'Warehouse Pincode_x',
       'Customer Pincode', 'Type of Shipment',
       'Warehouse Pincode_y', 'Forward Fixed Charge',
       'Forward Additional Weight Slab Charge', 'RTO Fixed Charge',
       'RTO Additional Weight Slab Charge'], inplace = True)

In [34]:
order_courier.rename(columns = {'Billing Amount (Rs.)' : 'Charges Billed by Courier Company (Rs.)', 'Delivery Zone charged by Courier Company' : 'Zone' }, inplace  = True)
order_courier.head()

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Delivery Zone as per X,Total weight as per Courier Company (KG),Zone,Charges Billed by Courier Company (Rs.),Expected Charge as per X (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.22,0.5,b,2.92,b,174.5,33.0,-141.5
1,2001806233,1091117222135,0.24,0.5,b,0.78,b,61.3,33.0,-28.3
2,2001806304,1091117222360,0.50,0.5,b,0.71,d,90.2,33.0,-57.2
3,2001810549,1091117806263,0.99,0.5,b,1.86,d,191.8,61.3,-130.5
4,2001811306,1091117958163,0.94,0.5,b,1.10,d,135.0,61.3,-73.7


In [35]:
# merge the order courier and weight slab to calculate the weight slab according to courier company

output_data_1 = pd.merge(left = order_courier, 
                         right = weight_slab,
                         how= 'inner',
                         on = 'Zone')

In [36]:
output_data_1.head(3)

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Delivery Zone as per X,Total weight as per Courier Company (KG),Zone,Charges Billed by Courier Company (Rs.),Expected Charge as per X (Rs.),Difference Between Expected Charges and Billed Charges (Rs.),Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,2001806210,1091117221940,0.22,0.5,b,2.92,b,174.5,33.0,-141.5,0.5,33.0,28.3,20.5,28.3
1,2001806233,1091117222135,0.24,0.5,b,0.78,b,61.3,33.0,-28.3,0.5,33.0,28.3,20.5,28.3
2,2001806458,1091117223244,0.70,0.5,b,1.00,b,61.3,61.3,0.0,0.5,33.0,28.3,20.5,28.3


In [37]:
# Drop some unwanted columns

output_data_1.drop(columns = ['Forward Fixed Charge',
                              'Forward Additional Weight Slab Charge', 'RTO Fixed Charge',
                              'RTO Additional Weight Slab Charge'], axis = 1, inplace = True)

In [38]:
output_data_1.rename(columns = {'Zone' : 'Delivery Zone charged by Courier Company'}, inplace = True)

seventh_column = output_data_1.pop('Weight Slabs')
output_data_1.insert(6, 'Weight slab charged by Courier Company (KG)', seventh_column)

eighth_column = output_data_1.pop('Expected Charge as per X (Rs.)')
output_data_1.insert(8, 'Expected Charge as per X (Rs.)', eighth_column)

ninth_column = output_data_1.pop('Delivery Zone as per X')
output_data_1.insert(6, 'Delivery Zone as per X', ninth_column)

# Output

In [39]:
# Create the summary table

dict_ = {"Metrics" : ["Total orders where X has been correctly charged", "Total Orders where X has been overcharged" , "Total Orders where X has been undercharged"],
         "Count" : [count_equal, count_over, count_under],
         "Amount" : [total_equal_invoice_amount, total_over_invoice_amount, total_under_invoice_amount]}


output_data_2 = pd.DataFrame(dict_)

In [40]:
output_data_1.sample(10)

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
65,2001811466,1091118001865,0.61,0.50,0.80,1.25,b,d,61.3,90.2,-28.9
70,2001813009,1091118553701,0.72,0.50,1.00,1.25,b,d,61.3,92.2,-30.9
59,2001810697,1091117807140,0.61,0.50,2.27,1.25,b,d,61.3,224.6,-163.3
8,2001806885,1091117229776,0.84,0.50,1.00,0.50,b,b,61.3,61.3,0.0
14,2001808507,1091117436383,0.61,0.50,0.79,0.50,b,b,61.3,61.3,0.0
110,2001808102,1091117435005,1.16,1.25,1.28,1.25,d,d,45.4,135.0,-89.6
41,2001807981,1091117327570,0.50,0.50,0.50,1.25,b,d,33.0,53.4,-20.4
106,2001807930,1091117327695,0.24,1.25,0.15,1.25,d,d,45.4,45.4,0.0
5,2001806801,1091117227816,0.36,0.50,1.35,0.50,b,b,33.0,89.6,-56.6
77,2001820978,1091120922803,0.52,0.50,0.50,1.25,b,d,61.3,45.4,15.9


In [42]:
output_data_2

,Metrics,Count,Amount
0,Total orders where X has been correctly charged,14,863.6
1,Total Orders where X has been overcharged,103,-6696.3
2,Total Orders where X has been undercharged,7,103.2


In [46]:
output_data_1.to_excel('Order level calculation.xlsx', sheet_name='Results', index = False)

In [48]:
output_data_2.to_excel('Summary table.xlsx', sheet_name='Summary', index = False)